# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import layers, datasets

## Hyperparameter Tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')
test_paths = glob('dataset/cifar/test/*.png')

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)
class_names

array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog',
       'horse', 'ship', 'truck'], dtype='<U10')

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    
    # 이번에는 onehot이 아닌 label 번호로
    return tf.argmax(onehot)

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale !!!
    
    label = get_label(path)
    
    return image, label

In [10]:
# transform images
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Learning Rate Scheduler

In [14]:
import math

In [15]:
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10 - epoch))

learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

## Training

In [17]:
# When providing an infinite dataset, you must specify the number of steps to run.
train_steps = len(train_paths) // batch_size
test_steps = len(test_paths) // batch_size

model.fit(train_dataset,
          validation_data=test_dataset,
          steps_per_epoch=train_steps,
          validation_steps=test_steps,
          epochs=num_epochs,
          callbacks=[learning_rate_scheduler]
)

Epoch 1/20
1562/1562 [==============================] - 14s 9ms/step - loss: 1.7098 - accuracy: 0.3606 - val_loss: 1.4735 - val_accuracy: 0.4576 - lr: 0.0010
Epoch 2/20
1562/1562 [==============================] - 24s 15ms/step - loss: 1.4183 - accuracy: 0.4816 - val_loss: 1.2947 - val_accuracy: 0.5270 - lr: 0.0010
Epoch 3/20
1562/1562 [==============================] - 22s 14ms/step - loss: 1.3243 - accuracy: 0.5200 - val_loss: 1.2194 - val_accuracy: 0.5633 - lr: 0.0010
Epoch 4/20
1562/1562 [==============================] - 22s 14ms/step - loss: 1.2641 - accuracy: 0.5442 - val_loss: 1.1660 - val_accuracy: 0.5742 - lr: 0.0010
Epoch 5/20
1562/1562 [==============================] - 21s 14ms/step - loss: 1.2243 - accuracy: 0.5563 - val_loss: 1.1149 - val_accuracy: 0.6048 - lr: 0.0010
Epoch 6/20
1562/1562 [==============================] - 22s 14ms/step - loss: 1.1897 - accuracy: 0.5728 - val_loss: 1.1107 - val_accuracy: 0.6020 - lr: 0.0010
Epoch 7/20
1562/1562 [=========================

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup